![Memories Teach](https://lh3.google.com/u/2/d/11o4JyBYhHcmz-LSt63GpOs4lfW9j39T_=w1912-h954-iv1)


`Basic to Advance in Google colab's for image processing, pattern recognition and computer vision`

[Phonepaserth SISAYKEO]

Reference: visioncolab

In [ ]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16427 sha256=17deeb47f495617a4936abc088a3f5a58f0dafe8e2b540af7f62e461a3d5a559
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [ ]:
import pandas as pd
import numpy as np
import os
import time
import warnings
warnings.filterwarnings('ignore')
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets
import torchvision.transforms as transforms
import copy
import random
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 7]

from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from efficientnet_pytorch import EfficientNet
import gc

# Functions

In [ ]:
def accuracy(predictions, labels):
    #round predictions to the closest integer
    probs = torch.softmax(predictions, dim=1)
    winners = probs.argmax(dim=1)
    corrects = (winners == labels).float()
    acc = corrects.sum() / len(corrects)
    return acc

def train_epoch(model, iterator, optimizer, criterion, device):
    epoch_loss = 0
    epoch_acc  = 0
    model.train()
    for batch in iterator:

        optimizer.zero_grad()

        img, label = batch
        img = img.to(device)
        label = label.to(device)

        predictions = model(img)#.squeeze()

        loss = criterion(predictions, label)
        acc  = accuracy(predictions, label)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc  += acc.item()

    return epoch_loss/len(iterator), epoch_acc/len(iterator)

def eval_epoch(model, iterator, criterion, device):

    #initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    #deactivating dropout layers
    model.eval()

    #deactivates autograd
    with torch.no_grad():
        for batch in iterator:

            img, label = batch
            img = img.to(device)
            label = label.to(device)

            #convert to 1d tensor
            predictions = model(img)

            #compute loss and accuracy
            loss = criterion(predictions, label)
            acc = accuracy(predictions, label)

            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def testing(model, iterator, batch_size, device):

    preds = []
    lbls  = []
    predictions = torch.Tensor(len(iterator)*batch_size).to(device)
    labels      = torch.Tensor(len(iterator)*batch_size).to(device)


    #deactivating dropout layers
    model.eval()

    #deactivates autograd
    with torch.no_grad():
        for batch in iterator:

            img, label = batch
            img = img.to(device)
            label = label.to(device)

            #convert to 1d tensor
            scores = model(img)
            probs = torch.softmax(scores, dim=1)
            winners = probs.argmax(dim=1)
            preds.append(winners)
            lbls.append(label)

    torch.cat(preds, out=predictions)
    torch.cat(lbls, out=labels)

    return predictions.cpu().numpy(), labels.cpu().numpy()

def metrics(predictions, labels):
    cm = confusion_matrix(labels, predictions)
    Accuracy = np.sum(np.diag(cm))/np.sum(cm)
    Pr = np.mean(np.diag(cm) / np.sum(cm, axis = 0))
    Re = np.mean(np.diag(cm) / np.sum(cm, axis = 1))
    F1 = 2*(Pr*Re)/(Pr + Re)
    return Accuracy, Pr, Re, F1, cm

def num_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
def train(model, train_dataloader, val_dataloader, optimizer, criterion, device, num_epochs,
          saved_models_folder, saved_scores_folder, save_path, printfreq=1):

    train_loss_hist = []
    train_acc_hist  = []
    val_loss_hist   = []
    val_acc_hist    = []

    best_val_loss = float('inf')
    best_model_wts = copy.deepcopy(model.state_dict())

    t0 = time.time()
    print(' Epoch    Train Loss    Val Loss    Train Acc    Val Acc    Best    Time [min]')
    print('-'*79)

    for epoch in range(num_epochs):
        t1 = time.time()
        st = '        '

        # Training metrics
        train_loss, train_acc = train_epoch(model, train_dataloader, optimizer, criterion, device)
        train_loss_hist.append(train_loss)
        train_acc_hist.append(train_acc)

        # Validation metrics
        val_loss, val_acc     = eval_epoch(model, val_dataloader, criterion, device)
        val_loss_hist.append(val_loss)
        val_acc_hist.append(val_acc)

        # Update best model
        if val_loss < best_val_loss:
            st = '     ***'
            best_val_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, os.path.join(saved_models_folder, 'best_' + save_path))

        if (epoch + 1) % printfreq == 0:
            t2 = (time.time() - t1)/60
            s = f'{epoch+1:6}{train_loss:12.4f}{val_loss:13.4f}{train_acc:13.4f}{val_acc:12.4f}{st}{t2:10.1f}'
            print(s)

        # Save current model and training information
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': val_loss,
        }, os.path.join(saved_models_folder, save_path))

        # Save the loss and accuracy values
        np.save(os.path.join(saved_scores_folder, f'{save_path}_train_loss'), train_loss_hist)
        np.save(os.path.join(saved_scores_folder, f'{save_path}_train_acc'), train_acc_hist)
        np.save(os.path.join(saved_scores_folder, f'{save_path}_val_loss'), val_loss_hist)
        np.save(os.path.join(saved_scores_folder, f'{save_path}_val_acc'), val_acc_hist)

    tfinal = (time.time() - t0)/60
    print('-'*79)
    print(f'Total time [min] for {num_epochs} Epochs: {tfinal:.1f}')
    return

In [ ]:
def display_train_metrics(saved_scores_folder, save_path):

    # Loss
    train_loss_hist = np.load(os.path.join(saved_scores_folder, f'{save_path}_train_loss.npy'))
    val_loss_hist = np.load(os.path.join(saved_scores_folder, f'{save_path}_val_loss.npy'))

    # Acc
    train_acc_hist = np.load(os.path.join(saved_scores_folder, f'{save_path}_train_acc.npy'))
    val_acc_hist = np.load(os.path.join(saved_scores_folder, f'{save_path}_val_acc.npy'))

    num_epochs = len(train_loss_hist)
    epochs = range(1, num_epochs + 1)


    plt.plot(epochs, train_loss_hist, marker='o', label='Training')
    plt.plot(epochs, val_loss_hist, marker='o', label='Validation')
    plt.title('Loss')
    plt.legend()
    plt.show()

    plt.plot(epochs, train_acc_hist, marker='o', label='Training')
    plt.plot(epochs, val_acc_hist, marker='o', label='Validation')
    plt.title('Accuracy')
    plt.legend()
    plt.show()

In [ ]:
def reset_model(model):
    for layer in model.children():
        if hasattr(layer, 'reset_parameters'):
            layer.reset_parameters()

# Model

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def initialize_model(model_name, num_classes, feature_extract=False, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model = None
    input_size = 0

    if 'resnet' in model_name:
        if model_name == 'resnet18':
            """ Resnet18"""
            model = models.resnet18(pretrained=use_pretrained)

        if model_name == 'resnet34':
            """ Resnet34"""
            model = models.resnet34(pretrained=use_pretrained)
        if model_name == 'resnet50':
            """ Resnet50"""
            model = models.resnet50(pretrained=use_pretrained)
        if model_name == 'resnet101':
            """ Resnet101"""
            model = models.resnet101(pretrained=use_pretrained)
        if model_name == 'resnet152':
            """ Resnet152"""
            model = models.resnet152(pretrained=use_pretrained)

        set_parameter_requires_grad(model, feature_extract)
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif 'effnet' in model_name:
        model_n = model_name[-1]
        full_model_name = f'efficientnet-b{model_n}'
        model = EfficientNet.from_pretrained(full_model_name, num_classes=num_classes)
        input_size = EfficientNet.get_image_size(full_model_name)


    elif model_name == "alexnet":
        """ Alexnet
        """
        model = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model, feature_extract)
        num_ftrs = model.classifier[6].in_features
        model.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    else:
        print("Invalid model name, exiting...")
        exit()

    return model, input_size

# Dataset

In [ ]:
# skin lesions: 7 classes
!wget https://www.dropbox.com/s/nzrvuoos7sgl5dh/exp4val.zip
!unzip -qq exp4val.zip
fpath = ''
num_classes =  7

--2023-07-19 19:49:31--  https://www.dropbox.com/s/nzrvuoos7sgl5dh/exp4val.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/nzrvuoos7sgl5dh/exp4val.zip [following]
--2023-07-19 19:49:32--  https://www.dropbox.com/s/raw/nzrvuoos7sgl5dh/exp4val.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc68051f10bf1a6f975227acb35e.dl.dropboxusercontent.com/cd/0/inline/CAKsrlKIjrlljBwhI9LEpjlQovsc-ZfSmp6S9Sfq4SZO96uHmLGsE2rndy3FEn0MlPMBDJpkgEbAZjsktG2Kgy88SfMwuYiJulvYAI6seHVMdAsjBrCb8sJLe5vuHfoSPTx7bauq68j5_5xg4pnWHX6Gk0YsHBZlh6Y3D0AOZCqbAg/file# [following]
--2023-07-19 19:49:32--  https://uc68051f10bf1a6f975227acb35e.dl.dropboxusercontent.com/cd/0/inline/CAKsrlKIjrlljBwhI9LEpjlQovsc-ZfSmp6S9Sfq4SZO96uHmLGsE2rndy3FEn0MlPMBDJpkgEbAZjsk

In [ ]:
def get_dataloaders(data_dir, input_size, batch_size=32):
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(input_size),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'test': transforms.Compose([
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        }
    print("Initializing Datasets and Dataloaders...")

    # Create training and validation datasets
    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val', 'test']}
    # Create training and validation dataloaders
    dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=2) for x in ['train', 'val', 'test']}
    return dataloaders_dict


# Training

In [ ]:
data_dir = ''
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [ ]:
model_name = 'effnetb0'
model, input_size = initialize_model(model_name, 7)

print(model)

p = count_parameters(model)

print(f'{p/1e6:8.4f}')


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:01<00:00, 18.0MB/s]


Loaded pretrained weights for efficientnet-b0
EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kern

## b0

In [ ]:
model_name = 'effnetb0'

saved_models_folder = 'saved_models_' + model_name
saved_scores_folder = 'saved_scores_' + model_name

try:
    os.mkdir(saved_models_folder)
    os.mkdir(saved_scores_folder)
except OSError:
    print('Folders already exist!')
    pass

model, input_size = initialize_model(model_name, 7)

dataloaders_dict = get_dataloaders(data_dir, input_size, batch_size=32)

Loaded pretrained weights for efficientnet-b0
Initializing Datasets and Dataloaders...


In [ ]:
model

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
    

In [ ]:
model = model.to(device)
num_epochs = 10
save_path = '0'

# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
train(model, dataloaders_dict['train'], dataloaders_dict['val'], optimizer, criterion, device, num_epochs,
      saved_models_folder, saved_scores_folder, save_path, printfreq=1)

 Epoch    Train Loss    Val Loss    Train Acc    Val Acc    Best    Time [min]
-------------------------------------------------------------------------------
     1      1.6480       1.4083       0.4205      0.5279     ***       1.0
     2      1.1927       1.0315       0.5823      0.6486     ***       0.9
     3      0.9955       0.9102       0.6409      0.6866     ***       0.9
     4      0.8806       0.8146       0.6757      0.7120     ***       0.9
     5      0.8101       0.7579       0.7020      0.7276     ***       0.9
     6      0.7463       0.7713       0.7287      0.7212               0.9
     7      0.6984       0.7172       0.7431      0.7464     ***       0.9
     8      0.6691       0.6744       0.7596      0.7678     ***       0.9
     9      0.6303       0.6629       0.7706      0.7684     ***       0.9
    10      0.6090       0.6800       0.7738      0.7710               0.9
-------------------------------------------------------------------------------
Total time 

## b1

In [ ]:
model_name = 'effnetb1'

saved_models_folder = 'saved_models_' + model_name
saved_scores_folder = 'saved_scores_' + model_name

try:
    os.mkdir(saved_models_folder)
    os.mkdir(saved_scores_folder)
except OSError:
    print('Folders already exist!')
    pass

model, input_size = initialize_model(model_name, 7)

dataloaders_dict = get_dataloaders(data_dir, input_size, batch_size=32)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b1-f1951068.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b1-f1951068.pth


  0%|          | 0.00/30.1M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b1
Initializing Datasets and Dataloaders...


In [ ]:
model = model.to(device)
num_epochs = 50
save_path = '0'

# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
train(model, dataloaders_dict['train'], dataloaders_dict['val'], optimizer, criterion, device, num_epochs,
      saved_models_folder, saved_scores_folder, save_path, printfreq=1)

 Epoch    Train Loss    Val Loss    Train Acc    Val Acc    Best    Time [min]
-------------------------------------------------------------------------------
     1      1.6872       1.4541       0.3995      0.5154     ***       1.5
     2      1.2131       1.0042       0.5862      0.6595     ***       1.5
     3      0.9797       0.8652       0.6511      0.7111     ***       1.5
     4      0.8628       0.7776       0.6903      0.7297     ***       1.5
     5      0.7795       0.7019       0.7165      0.7525     ***       1.5
     6      0.7178       0.7138       0.7388      0.7509               1.5
     7      0.6774       0.6535       0.7542      0.7790     ***       1.5
     8      0.6310       0.6369       0.7697      0.7737     ***       1.5
     9      0.6007       0.6400       0.7790      0.7799               1.5
    10      0.5569       0.6173       0.7990      0.7917     ***       1.5
    11      0.5315       0.5986       0.8069      0.8076     ***       1.5
    12      0.49

## b2

In [ ]:
model_name = 'effnetb2'

saved_models_folder = 'saved_models_' + model_name
saved_scores_folder = 'saved_scores_' + model_name

try:
    os.mkdir(saved_models_folder)
    os.mkdir(saved_scores_folder)
except OSError:
    print('Folders already exist!')
    pass

model, input_size = initialize_model(model_name, 7)

dataloaders_dict = get_dataloaders(data_dir, input_size, batch_size=32)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b2-8bb594d6.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b2-8bb594d6.pth


  0%|          | 0.00/35.1M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b2
Initializing Datasets and Dataloaders...


In [ ]:
model = model.to(device)
num_epochs = 50
save_path = '0'

# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
train(model, dataloaders_dict['train'], dataloaders_dict['val'], optimizer, criterion, device, num_epochs,
      saved_models_folder, saved_scores_folder, save_path, printfreq=1)

 Epoch    Train Loss    Val Loss    Train Acc    Val Acc    Best    Time [min]
-------------------------------------------------------------------------------
     1      1.7150       1.5511       0.3877      0.4938     ***       2.0
     2      1.2605       1.0795       0.5694      0.6412     ***       2.0
     3      1.0269       0.8568       0.6350      0.6972     ***       2.0
     4      0.8977       0.7670       0.6720      0.7163     ***       2.0
     5      0.7963       0.7008       0.7112      0.7485     ***       2.0
     6      0.7341       0.7009       0.7284      0.7477               2.0
     7      0.6788       0.6446       0.7541      0.7524     ***       2.0
     8      0.6297       0.6199       0.7665      0.7750     ***       2.0
     9      0.6093       0.6111       0.7738      0.7811     ***       2.0
    10      0.5616       0.6072       0.7958      0.7821     ***       2.0
    11      0.5350       0.5892       0.8063      0.7897     ***       2.0
    12      0.49

## b3

In [ ]:
model_name = 'effnetb3'

saved_models_folder = 'saved_models_' + model_name
saved_scores_folder = 'saved_scores_' + model_name

try:
    os.mkdir(saved_models_folder)
    os.mkdir(saved_scores_folder)
except OSError:
    print('Folders already exist!')
    pass

model, input_size = initialize_model(model_name, 7)

dataloaders_dict = get_dataloaders(data_dir, input_size, batch_size=32)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b3-5fb5a3c3.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b3-5fb5a3c3.pth


  0%|          | 0.00/47.1M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b3
Initializing Datasets and Dataloaders...


In [ ]:
model = model.to(device)
num_epochs = 50
save_path = '0'

# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
train(model, dataloaders_dict['train'], dataloaders_dict['val'], optimizer, criterion, device, num_epochs,
      saved_models_folder, saved_scores_folder, save_path, printfreq=1)

 Epoch    Train Loss    Val Loss    Train Acc    Val Acc    Best    Time [min]
-------------------------------------------------------------------------------
     1      1.6771       1.4356       0.4039      0.5028     ***       3.3
     2      1.2073       1.0184       0.5692      0.6297     ***       3.3
     3      1.0112       0.8704       0.6322      0.6767     ***       3.3
     4      0.8800       0.7757       0.6777      0.7172     ***       3.3
     5      0.7657       0.7640       0.7243      0.7226     ***       3.3
     6      0.7119       0.7111       0.7379      0.7601     ***       3.3
     7      0.6502       0.6466       0.7602      0.7769     ***       3.3
     8      0.6037       0.6461       0.7843      0.7657     ***       3.3
     9      0.5564       0.6192       0.8017      0.7857     ***       3.3
    10      0.5192       0.6787       0.8113      0.7690               3.3
    11      0.4824       0.5928       0.8265      0.7992     ***       3.3
    12      0.45

## b4

In [ ]:
model_name = 'effnetb4'

saved_models_folder = 'saved_models_' + model_name
saved_scores_folder = 'saved_scores_' + model_name

try:
    os.mkdir(saved_models_folder)
    os.mkdir(saved_scores_folder)
except OSError:
    print('Folders already exist!')
    pass

model, input_size = initialize_model(model_name, 7)

dataloaders_dict = get_dataloaders(data_dir, input_size, batch_size=16)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b4-6ed6700e.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b4-6ed6700e.pth


  0%|          | 0.00/74.4M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b4
Initializing Datasets and Dataloaders...


In [ ]:
model = model.to(device)
num_epochs = 30
save_path = '0'

# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
train(model, dataloaders_dict['train'], dataloaders_dict['val'], optimizer, criterion, device, num_epochs,
      saved_models_folder, saved_scores_folder, save_path, printfreq=1)

 Epoch    Train Loss    Val Loss    Train Acc    Val Acc    Best    Time [min]
-------------------------------------------------------------------------------
     1      1.5851       1.1609       0.4387      0.6356     ***       6.8
     2      1.1104       0.8635       0.5949      0.7064     ***       6.8
     3      0.9189       0.8299       0.6576      0.7136     ***       6.8
     4      0.8105       0.7236       0.7070      0.7384     ***       6.8
     5      0.7202       0.7053       0.7337      0.7511     ***       6.8
     6      0.6628       0.6357       0.7519      0.7805     ***       6.8
     7      0.6217       0.6401       0.7741      0.7720               6.8
     8      0.5618       0.5808       0.7934      0.8095     ***       6.8
     9      0.5182       0.6517       0.8096      0.7657               6.8
    10      0.4786       0.5776       0.8308      0.8061     ***       6.8
    11      0.4440       0.5940       0.8391      0.7994               6.8
    12      0.42

## b5

In [ ]:

gc.collect()

torch.cuda.empty_cache()

In [ ]:
model_name = 'effnetb5'

saved_models_folder = 'saved_models_' + model_name
saved_scores_folder = 'saved_scores_' + model_name

try:
    os.mkdir(saved_models_folder)
    os.mkdir(saved_scores_folder)
except OSError:
    print('Folders already exist!')
    pass

model, input_size = initialize_model(model_name, 7)

dataloaders_dict = get_dataloaders(data_dir, input_size, batch_size=10)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b5-b6417697.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b5-b6417697.pth


  0%|          | 0.00/117M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b5
Initializing Datasets and Dataloaders...


In [ ]:
model = model.to(device)
num_epochs = 30
save_path = '0'

# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
train(model, dataloaders_dict['train'], dataloaders_dict['val'], optimizer, criterion, device, num_epochs,
      saved_models_folder, saved_scores_folder, save_path, printfreq=1)

 Epoch    Train Loss    Val Loss    Train Acc    Val Acc    Best    Time [min]
-------------------------------------------------------------------------------
     1      1.5055       1.0902       0.4569      0.6390     ***      14.1
     2      1.0261       0.7834       0.6205      0.7267     ***      14.1
     3      0.8620       0.6988       0.6801      0.7419     ***      14.1
     4      0.7624       0.6511       0.7182      0.7790     ***      14.1
     5      0.6813       0.6083       0.7541      0.7924     ***      14.1
     6      0.6096       0.5880       0.7800      0.7800     ***      14.1
     7      0.5548       0.5773       0.7971      0.8000     ***      14.1
     8      0.5080       0.5512       0.8216      0.8143     ***      14.1
     9      0.4663       0.6177       0.8310      0.7895              14.1
    10      0.4272       0.5530       0.8505      0.8133              14.1
    11      0.4102       0.5596       0.8539      0.8171              14.1
    12      0.39

## b6

In [ ]:
gc.collect()
torch.cuda.empty_cache()

model_name = 'effnetb6'

saved_models_folder = 'saved_models_' + model_name
saved_scores_folder = 'saved_scores_' + model_name

try:
    os.mkdir(saved_models_folder)
    os.mkdir(saved_scores_folder)
except OSError:
    print('Folders already exist!')
    pass

model, input_size = initialize_model(model_name, 7)

dataloaders_dict = get_dataloaders(data_dir, input_size, batch_size=5)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b6-c76e70fd.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b6-c76e70fd.pth


  0%|          | 0.00/165M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b6
Initializing Datasets and Dataloaders...


In [ ]:
model = model.to(device)
num_epochs = 30
save_path = '0'

# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
train(model, dataloaders_dict['train'], dataloaders_dict['val'], optimizer, criterion, device, num_epochs,
      saved_models_folder, saved_scores_folder, save_path, printfreq=1)

# Testing

In [ ]:
data_dir = '../exp4val'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 32

## b0

In [ ]:
gc.collect()
torch.cuda.empty_cache()

model_name = 'effnetb0'
save_path = 'best_0'

saved_models_folder = 'saved_models_' + model_name
saved_scores_folder = 'saved_scores_' + model_name

model, input_size = initialize_model(model_name, 7)
model = model.to(device)

dataloaders_dict = get_dataloaders(data_dir, input_size, batch_size=32)

model.load_state_dict(torch.load(os.path.join(saved_models_folder, save_path)))


Loaded pretrained weights for efficientnet-b0
Initializing Datasets and Dataloaders...


<All keys matched successfully>

In [ ]:
predictions, labels = testing(model, dataloaders_dict['test'], 32, device)
Accuracy, Pr, Re, F1, cm = metrics(predictions, labels)

print('Accuracy  = ' + str(Accuracy))
print('Precision = ' + str(Pr))
print('Recall    = ' + str(Re))
print('F1        = ' + str(F1))
print(cm)

Accuracy  = 0.7619047619047619
Precision = 0.7652535636406606
Recall    = 0.7619047619047619
F1        = 0.7635754910928079
[[14  8  3  1  4  0  0]
 [ 1 25  0  2  2  0  0]
 [ 2  1 18  1  6  2  0]
 [ 1  0  1 26  1  1  0]
 [ 0  1  4  1 19  5  0]
 [ 0  1  0  0  1 28  0]
 [ 0  0  0  0  0  0 30]]


## b1

In [ ]:
gc.collect()
torch.cuda.empty_cache()

model_name = 'effnetb1'
save_path = 'best_0'

saved_models_folder = 'saved_models_' + model_name
saved_scores_folder = 'saved_scores_' + model_name

model, input_size = initialize_model(model_name, 7)
model = model.to(device)

dataloaders_dict = get_dataloaders(data_dir, input_size, batch_size=32)

model.load_state_dict(torch.load(os.path.join(saved_models_folder, save_path)))


In [ ]:
predictions, labels = testing(model, dataloaders_dict['test'], 32, device)
Accuracy, Pr, Re, F1, cm = metrics(predictions, labels)

print('Accuracy  = ' + str(Accuracy))
print('Precision = ' + str(Pr))
print('Recall    = ' + str(Re))
print('F1        = ' + str(F1))
print(cm)

## b2

In [ ]:
gc.collect()
torch.cuda.empty_cache()

model_name = 'effnetb2'
save_path = 'best_0'

saved_models_folder = 'saved_models_' + model_name
saved_scores_folder = 'saved_scores_' + model_name

model, input_size = initialize_model(model_name, 7)
model = model.to(device)

dataloaders_dict = get_dataloaders(data_dir, input_size, batch_size=32)

model.load_state_dict(torch.load(os.path.join(saved_models_folder, save_path)))


In [ ]:
predictions, labels = testing(model, dataloaders_dict['test'], 32, device)
Accuracy, Pr, Re, F1, cm = metrics(predictions, labels)

print('Accuracy  = ' + str(Accuracy))
print('Precision = ' + str(Pr))
print('Recall    = ' + str(Re))
print('F1        = ' + str(F1))
print(cm)

## b3

In [ ]:
gc.collect()
torch.cuda.empty_cache()

model_name = 'effnetb3'
save_path = 'best_0'

saved_models_folder = 'saved_models_' + model_name
saved_scores_folder = 'saved_scores_' + model_name

model, input_size = initialize_model(model_name, 7)
model = model.to(device)

dataloaders_dict = get_dataloaders(data_dir, input_size, batch_size=32)

model.load_state_dict(torch.load(os.path.join(saved_models_folder, save_path)))


In [ ]:
predictions, labels = testing(model, dataloaders_dict['test'], 32, device)
Accuracy, Pr, Re, F1, cm = metrics(predictions, labels)

print('Accuracy  = ' + str(Accuracy))
print('Precision = ' + str(Pr))
print('Recall    = ' + str(Re))
print('F1        = ' + str(F1))
print(cm)

## b4

In [ ]:
gc.collect()
torch.cuda.empty_cache()

model_name = 'effnetb4'
save_path = 'best_0'

saved_models_folder = 'saved_models_' + model_name
saved_scores_folder = 'saved_scores_' + model_name

model, input_size = initialize_model(model_name, 7)
model = model.to(device)

dataloaders_dict = get_dataloaders(data_dir, input_size, batch_size=16)

model.load_state_dict(torch.load(os.path.join(saved_models_folder, save_path)))


In [ ]:
predictions, labels = testing(model, dataloaders_dict['test'], 16, device)
Accuracy, Pr, Re, F1, cm = metrics(predictions, labels)

print('Accuracy  = ' + str(Accuracy))
print('Precision = ' + str(Pr))
print('Recall    = ' + str(Re))
print('F1        = ' + str(F1))
print(cm)

## b5

In [ ]:
gc.collect()
torch.cuda.empty_cache()

model_name = 'effnetb5'
save_path = 'best_0'

saved_models_folder = 'saved_models_' + model_name
saved_scores_folder = 'saved_scores_' + model_name

model, input_size = initialize_model(model_name, 7)
model = model.to(device)

dataloaders_dict = get_dataloaders(data_dir, input_size, batch_size=10)

model.load_state_dict(torch.load(os.path.join(saved_models_folder, save_path)))


In [ ]:
predictions, labels = testing(model, dataloaders_dict['test'], 10, device)
Accuracy, Pr, Re, F1, cm = metrics(predictions, labels)

print('Accuracy  = ' + str(Accuracy))
print('Precision = ' + str(Pr))
print('Recall    = ' + str(Re))
print('F1        = ' + str(F1))
print(cm)

## b6

In [ ]:
gc.collect()
torch.cuda.empty_cache()

model_name = 'effnetb6'
save_path = 'best_0'

saved_models_folder = 'saved_models_' + model_name
saved_scores_folder = 'saved_scores_' + model_name

model, input_size = initialize_model(model_name, 7)
model = model.to(device)

dataloaders_dict = get_dataloaders(data_dir, input_size, batch_size=5)

model.load_state_dict(torch.load(os.path.join(saved_models_folder, save_path)))


In [ ]:
predictions, labels = testing(model, dataloaders_dict['test'], 5, device)
Accuracy, Pr, Re, F1, cm = metrics(predictions, labels)

print('Accuracy  = ' + str(Accuracy))
print('Precision = ' + str(Pr))
print('Recall    = ' + str(Re))
print('F1        = ' + str(F1))
print(cm)